[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/cookbook/indoxArcg/Milvus_vectorStore.ipynb)

In [ ]:
!pip install indoxArcg
!pip install openai
!pip install pymilvus

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indoxArcg`:

### Windows

1. **Create the virtual environment:**
```bash
python -m venv indoxArcg
```
2. **Activate the virtual environment:**
```bash
indoxArcg\Scripts\activate
```

### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indoxArcg
```

2. **Activate the virtual environment:**
    ```bash
   source indoxArcg/bin/activate
```
### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
pip install -r requirements.txt
```


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
INDOX_API_KEY = os.getenv("INDOX_API_KEY")

## Initial Setup

The following imports are essential for setting up the Indox application. These imports include the main Indox retrieval augmentation module, question-answering models, embeddings, and data loader splitter.


In [ ]:
from indoxArcg.llms import IndoxApi
from indoxArcg.embeddings import IndoxApiEmbedding
from indoxArcg.data_loader_splitter import ClusteredSplit

In [4]:
openai_qa_indox = IndoxApi(api_key=INDOX_API_KEY)
embed_openai_indox = IndoxApiEmbedding(api_key=INDOX_API_KEY, model="text-embedding-3-small")


INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            
INFO: Initialized IndoxOpenAIEmbedding with model: text-embedding-3-small


In [5]:
#!wget https://raw.githubusercontent.com/osllmai/inDox/master/Demo/sample.txt

In [6]:
file_path = "sample.txt"
loader_splitter = ClusteredSplit(file_path=file_path, embeddings=embed_openai_indox, summary_model=openai_qa_indox)
docs = loader_splitter.load_and_chunk()

INFO: ClusteredSplit initialized successfully


In [ ]:
from indoxArcg.vector_stores import Milvus

## Vector Store Connection and Document Storage

In this step, we connect the Indox application to the vector store and store the processed documents.


In [ ]:
db = Milvus(collection_name="sample", embedding_model=embed_openai_indox)
db.add(docs=docs)

## Querying and Interpreting the Response

In this step, we query the Indox application with a specific question and use the QA model to get the response. 



In [12]:
query = "How cinderella reach her happy ending?"
from indoxArcg.pipelines.rag import RAG
retriever = RAG(llm=openai_qa_indox,vector_store=db,top_k= 5)
answer = retriever.infer(query)
answer

INFO: Retrieving context and scores from the vector database
INFO: Embedding documents
INFO: Starting to fetch embeddings texts using engine: text-embedding-3-small
INFO: Generating answer without document relevancy filter
INFO: Query answered successfully


"In the story of Cinderella, she reaches her happy ending through her kindness, resilience, and unwavering belief in goodness. Despite facing hardships and mistreatment from her stepfamily, Cinderella remains true to herself and never loses hope. With the help of her fairy godmother, she is able to attend the royal ball and capture the heart of the prince. Ultimately, Cinderella's pure heart and inner strength lead her to her happy ending, where she is rewarded for her virtues and finds love and happiness."